<a href="https://colab.research.google.com/github/syedfaiqueali/Kaggle-Notebooks/blob/main/deepseek-r1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.2 MB/s eta 0:00:00


# Imports

In [24]:
import gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import clear_output

# Helper Classes

In [59]:
class DeepSeekModel:
    def __init__(self, model_name: str = "DeepSeek-R1-Distill-Qwen-1.5B"):
        self.model_name = model_name
        self.model_id = f"deepseek-ai/{model_name}"
        self.tokenizer = None
        self.model = None

    def load_model(self):
        """Load the DeepSeek-R1 model and tokenizer."""
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            device_map="auto",
            torch_dtype=torch.float16
        )
        clear_output()

    def generate_prediction(self, prompt: str, max_new_tokens: int = 2500):
        """Generate a prediction based on the input prompt."""
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to("cuda")
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            # pad_token_id=self.tokenizer.eos_token_id  # Set pad_token_id to eos_token_id
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def unload_model(self):
        """Unload the model to free up resources."""
        del self.model
        del self.tokenizer
        gc.collect()
        torch.cuda.empty_cache()

In [60]:
class StockPredictor:
    def __init__(self, model_name: str):
        self.model = DeepSeekModel(model_name)

    def analyze_and_predict(self, csv_data_string: str):
        """Analyze stock data and predict future performance."""
        prompt = self.create_prompt(csv_data_string)
        self.model.load_model()
        prediction = self.model.generate_prediction(prompt)
        self.model.unload_model()
        return prediction

    def create_prompt(self, csv_data_string: str):
        """Create a prompt for the DeepSeek-R1 model."""
        prompt = f"""
A conversation between User and Assistant. The User provides daily stock data for a specific stock symbol in CSV format. The Assistant analyzes the data and predicts the stock's performance for the next 3-5 days.

User: Here is the daily stock data:

Date,Open,High,Low,Close,Volume
{csv_data_string}

Assistant: <think>
1. Understand the provided stock data.
2. Analyze historical trends and patterns.
3. Apply relevant financial models and indicators.
4. Predict the stock's performance for the next 3-5 days.
5. Consider external factors that might influence the stock.
6. Tell likelihood for your prediction, RSI value.
</think>
"""
        return prompt

# Code

In [61]:
# Load Model
predictor = StockPredictor(model_name="DeepSeek-R1-Distill-Qwen-1.5B")

In [62]:
csv_data = """
Date,Open,High,Low,Close,Volume
2025-01-01,100,105,99,104,1500000
2025-01-02,104,106,102,105,1200000
2025-01-03,105,107,103,106,1300000
2025-01-04,106,108,104,107,1400000
2025-01-05,107,109,105,108,1600000
"""

prediction = predictor.analyze_and_predict(csv_data_string=csv_data)
print("Prediction:\n", prediction)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Prediction:
 
A conversation between User and Assistant. The User provides daily stock data for a specific stock symbol in CSV format. The Assistant analyzes the data and predicts the stock's performance for the next 3-5 days.

User: Here is the daily stock data:

Date,Open,High,Low,Close,Volume

Date,Open,High,Low,Close,Volume
2025-01-01,100,105,99,104,1500000
2025-01-02,104,106,102,105,1200000
2025-01-03,105,107,103,106,1300000
2025-01-04,106,108,104,107,1400000
2025-01-05,107,109,105,108,1600000


Assistant: <think>
1. Understand the provided stock data.
2. Analyze historical trends and patterns.
3. Apply relevant financial models and indicators.
4. Predict the stock's performance for the next 3-5 days.
5. Consider external factors that might influence the stock.
6. Tell likelihood for your prediction, RSI value.
</think>
Certainly! To predict the stock's performance for the next 3-5 days, I'll analyze the historical data provided. Let's look at the data step by step:

1. **Date, Op

In [64]:
csv_data = """
Date,Open,High,Low,Close,Volume
2025-01-01,100,105,99,104,15
2025-01-02,104,106,102,105,1200000
2025-01-03,105,107,103,106,13
2025-01-04,106,108,104,107,1400000
2025-01-05,107,55,55,108,16
"""

prediction = predictor.analyze_and_predict(csv_data_string=csv_data)
print("Prediction:\n", prediction)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Prediction:
 
A conversation between User and Assistant. The User provides daily stock data for a specific stock symbol in CSV format. The Assistant analyzes the data and predicts the stock's performance for the next 3-5 days.

User: Here is the daily stock data:

Date,Open,High,Low,Close,Volume

Date,Open,High,Low,Close,Volume
2025-01-01,100,105,99,104,15
2025-01-02,104,106,102,105,1200000
2025-01-03,105,107,103,106,13
2025-01-04,106,108,104,107,1400000
2025-01-05,107,55,55,108,16


Assistant: <think>
1. Understand the provided stock data.
2. Analyze historical trends and patterns.
3. Apply relevant financial models and indicators.
4. Predict the stock's performance for the next 3-5 days.
5. Consider external factors that might influence the stock.
6. Tell likelihood for your prediction, RSI value.
</think>
Sure, let's analyze the provided stock data step by step to predict its performance for the next 3-5 days.

### Step 1: Understanding the Data
Here's the stock data provided:

```
